## Imports

In [ ]:
%load_ext autoreload
%autoreload

In [254]:
#from dipy.data import get_data
from nibabel import trackvis as tv
from dipy.segment.clustering import QuickBundles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import nibabel as nib
import networkx as nx
import collections
import os
# from scipy.stats import norm
from numpy import linspace
import scipy
import scipy.stats



## Actual stuff

In [280]:
def ts_to_graph(file, threshold):
    
    sub = nib.load(file)
    sub = pd.DataFrame(sub.get_data()).T

    cor = sub.corr() # correlation plot
   
    cor[cor == 1] = 0 # set the diagonal to 0
    adjmat = cor * 10 # spread out the values some
    adjmat[abs(adjmat) < threshold] = 0 # remove values with least correlation
    adjmat = np.floor(adjmat) # discretize; don't use for now but maybe later

    G = nx.from_numpy_matrix(adjmat.values)
    return(G, adjmat)
    

In [287]:
def deg_counter(G):
    deglist = [d for n, d in G.degree()]
    deg_seq = sorted(deglist, reverse=True) # degree histogram
    highest = deglist.index(max(deglist))
    degreeCount = collections.Counter(deg_seq)
    deg, cnt = zip(*degreeCount.items())
    count = pd.Series(cnt, index=deg)#, name = file[-19:-13])
    n_edges = len(list(G.edges))

    return(count, n_edges, highest)


In [277]:
def stats(df, edges):
    ''' takes dataframe and edgecount list, returns dict with series or values
        mdeg = mean # per deg, 
        sdeg = sd/deg, 
        medg = mean #edges, 
        sedg = sd #edges '''
    d = {}
    edges = pd.Series(edges)
    df.fillna(0)
    d["mn_deg"] = df.mean(axis=1)
    d["mn_deg"].name = "mean"
    d["sd_deg"] = df.std(axis=1).fillna(0)
    d["sd_deg"].name = "sd"
    d["mn_edge"] = edges.mean()
    d["sd_edge"] = edges.std()
    return(d)
    

In [290]:
def get_graph_avgs(n, d = 50, plots=0, writ = 0):
    poss = [15, 25, 50, 100]
    if d not in poss:
        print("Please choose 15, 25, 50, or 100 partition")
        return
    dstring = "C:/Users/Seth/Desktop/Internet Algorithms/NeuroNetworks/3T_HCP1200_MSMAll_d%d_ts2_Z/" % d
    
    df = pd.DataFrame()
    edges = []
    mean_adj = []
    fnames = []
    highest_deg_region = []
    for i, file in enumerate(os.listdir(dstring)):
        if i == n:
            break
        if not (i+1 % 10):
            print(i)
        filename = dstring + file
        g, am = ts_to_graph(filename,  1.5)
        gg, ec, h = deg_counter(g)
        edges.append(ec);highest_deg_region.append(h);mean_adj.append(am)
        df = pd.concat([df, gg], axis = 1)
        fnames.append(file)
    edgeCount = collections.Counter(edges)
    enumb, cnt = zip(*edgeCount.items())
    #amms = mean_adj.groupby([1,2]).sum()
    avg_edge = pd.concat(mean_adj)#, keys=fnames)
    #print(avg_edge)
    avg_adj = avg_edge.groupby(avg_edge.index).mean()
    if writ:
        avg_adj.to_csv("./meanadj.csv")
    
    if plots:
        plots.close()
        plt.bar(enumb, cnt)
        plt.xlabel("number of edges")
        plt.ylabel('graphs having number')
        plt.title('Histogram of edge counts for %d patients' %n)
        plt.grid()
        plt.savefig("../img/functional_edge_hist.pdf")
    #print(df)
    out_d = {'stats':stats(df, edges), 'meanNumEdges':enumb, 'countEdge':cnt, 'maxDeg':highest_deg_region}
    return(out_d)

In [291]:
out_d = get_graph_avgs(2, writ=1, plots=0)

In [292]:
print(out_d['stats'])

{'mn_deg': 0    26.5
1    15.5
2     4.0
3     1.5
4     2.0
8     1.0
Name: mean, dtype: float64, 'sd_deg': 0    6.363961
1    0.707107
2    2.828427
3    0.707107
4    1.414214
8    0.000000
Name: sd, dtype: float64, 'mn_edge': 20.0, 'sd_edge': 9.899494936611665}


In [274]:
def fit_n_edge_test(x, y):
    ''' x is number of edges
        y is count of '''
    size = len(y)
    dist_names = ['alpha', 'beta', 'arcsine', 'rayleigh']
    plt.close()
    plt.bar(x, y)
    plt.xlabel("number of edges")
    plt.ylabel('graphs having number')
    plt.title('Histogram of edge counts for %d patients' % size)
    plt.grid()
    for dist_name in dist_names:
        dist = getattr(scipy.stats, dist_name)
        param = dist.fit(y)
        pdf_fitted = dist.pdf(x, *param[:-2], loc=param[-2], scale=param[-1]) * size
        plt.plot(pdf_fitted, label=dist_name)
        plt.xlim(0,47)
    plt.legend(loc='upper left')
    plt.show()
    # picking 150 of from a normal distrubution
    # with mean 0 and standard deviation 1
    

In [269]:
fit_n_edge_test(nedges, count)

C:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:161: RuntimeWarning: The number of calls to function has reached maxfev = 600.
  warnings.warn(msg, RuntimeWarning)


## Junk code messing around with nii files

In [ ]:
from dipy.segment.clustering import QuickBundles as qbs

In [ ]:
# brn_avg = nib.load("./HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii")
# print(type(brn_avg.header))
# ba_data = brn_avg.get_data()
# print(type(ba_data))
# brn_avg_em = nib.load("./HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500_Eigenmaps.dtseries.nii")
# print(type(brn_avg_em))
# sub_100206_d50 = nib.load("./workbench/bin_windows64/100206.dconn.nii")
# sd50 = sub_100206_d50.get_data() 'paging file too small'; guess we have to work with cifti and not numpy...
# print(sd50.shape)
# print(type(sub_100206_d50))
# sub_100206_d50.shape

In [ ]:
#sub_100206_d50 = nib.load("./workbench/bin_windows64/100206.dtseries.nii")
#textract_100206_50 = pd.read_csv("100206.txt", sep=" ", header = None)#mac
#"HCP_PTN1200/node_timeseries/3T_HCP1200_MSMAll_d50_ts2/", sep = " ", header = None)
#sub_100206_d50.shape

In [ ]:
textract_100206_50 = pd.read_csv("./workbench/bin_windows64/100206_txt.txt", sep="\t", header=None) #pc
print(textract_100206_50.shape)
p100206_corr = textract_100206_50.corr()#np.fill_diagonal(textract_100206_50.corr(), 0)
print(p100206_corr.head())
# diagonal is perfectly correlated, cut that out

p100206_corr[p100206_corr == 1] = 0

In [ ]:
cax = plt.matshow(p100206_corr)
plt.colorbar(cax)
plt.show()
    # we're discretizing the correlation
adj_mat = p100206_corr * 10

# remove bottom 15% of edges, (arbitrarily?)
adj_mat[abs(adj_mat) < 1.5] = 0
# bucket them
adj_mat = np.floor(adj_mat)

cax = plt.matshow(adj_mat)
nm = mpc.Normalize(vmin=-4, vmax=4)
plt.colorbar(cax, norm=nm)
plt.show()

In [ ]:
qb = qbs(threshold=10)
clusters = qb.cluster(sd50)

In [ ]:
nd = nib.load("C:\Users\Seth\Desktop\Internet Algorithms\NeuroNetworks\HCP_PTN1200\node_timeseries\3T_HCP1200_MSMAll_d50_ts2")

In [ ]:
import dipy.reconst.dti as dti

In [ ]:
white_matter = (ba_data == 1) | (ba_data == 2)

In [ ]:
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model

csa_model = CsaOdfModel(gtab, sh_order=6)
csa_peaks = peaks_from_model(csa_model, data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=white_matter)

In [ ]:
# can't use igraph bc/ windows.
#import igraph as ig; G_100206 = ig.Graph.Adjacency((abs(adj_mat > 0)).tolist())

G_100206 = nx.from_numpy_matrix(adj_mat.values)#, create_using=nx.MultiGraph())
len(G_100206.edges())
#G_100206.edges(data=True)
#nx.diameter(G_100206) # #48 has weight under 10%
#comps = nx.connected_components(G_100206)
#[print(comp) for comp in comps]

#print(len(list(G_100206.edges())))

degree_sequence = sorted([d for n, d in G_100206.degree()], reverse=True)  # degree sequence
# print "Degree sequence", degree_sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

fig, ax = plt.subplots()
plt.bar(deg, cnt, width=0.80, color='b')

plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Degree")
ax.set_xticks([d + 0.4 for d in deg])
ax.set_xticklabels(deg)

# draw graph in inset
plt.axes([0.4, 0.4, 0.5, 0.5])
Gcc = sorted(nx.connected_component_subgraphs(G_100206), key=len, reverse=True)[0]
pos = nx.spring_layout(G_100206)
plt.axis('off')
nx.draw_networkx_nodes(G_100206, pos, node_size=20)
nx.draw_networkx_edges(G_100206, pos, alpha=0.4)

plt.show()
